# Genetic optimisation of SVC classifier for the SDSS data

## Load data

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import warnings
import helpers
from helpers import DataSet
import matplotlib as mpl
import os
import random

# Common imports
import pandas as pd
import numpy as np
import seaborn as sns

# Imports for ML
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style('whitegrid')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
# Ignore useless warnings (see SciPy issue #5998)
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATA_PATH = "Skyserver_SQL2_27_2018 6_51_39 PM.csv"
RESULTS_FOLDER = "results"

# We load the data. Those that have nothing to do with the features of the objects are ignored.
sdss_data = pd.read_csv(DATA_PATH)

# We have a general look at the features
sdss_data.head(3)

print(sdss_data.columns.values)
sdss_data.drop(['objid', 'run', 'rerun', 'camcol', 'field',
                'specobjid', 'fiberid', 'mjd', 'plate'], axis=1, inplace=True)
sdss_data.head(1)

sdss_df_fe = sdss_data

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

X = sdss_data.drop(['class'], axis=1)
y = sdss_data['class']

class_names = ["GALAXY", "QSO", "STAR"]

std_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test = std_scaler.fit_transform(X_test.astype(np.float64))
X_train = std_scaler.fit_transform(X_train.astype(np.float64))

['objid' 'ra' 'dec' 'u' 'g' 'r' 'i' 'z' 'run' 'rerun' 'camcol' 'field'
 'specobjid' 'class' 'redshift' 'plate' 'mjd' 'fiberid']


## Optimize

In [2]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV

paramgrid = {"C": np.logspace(-9, , num=25, base=10),
             "penalty": ["l1", "l2"],
             "loss": ["hinge", "squared_hinge"],
             "dual": [True, False],
             "multi_class": ["crammer_singer", "ovr"],
             "fit_intercept": [True, False],
             "intercept_scaling": np.logspace(-9, 9, num=25, base=10),
             "random_state": [42],
             "tol": [1e-5],
             "max_iter": np.logspace(2, 4, num=25, base=10)
             }

random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=XGBClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=5),
                                   verbose=3,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=15,
                                   n_jobs=8,
                                   error_score=0.0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = cv.fit(X, y)

Types [2, 1, 1, 1, 1, 1, 2, 1, 2, 2] and maxint [24, 1, 1, 1, 1, 1, 24, 0, 0, 24] detected


/home/ljosfer/anaconda3/envs/deep/lib/python3.6/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


[CV] C=1e-06, penalty=l1, loss=squared_hinge, dual=True, multi_class=ovr, fit_intercept=False, intercept_scaling=177.82794100389228, random_state=42, tol=1e-05, max_iter=177.82794100389228 
[CV] C=1e-09, penalty=l1, loss=hinge, dual=True, multi_class=ovr, fit_intercept=True, intercept_scaling=5.623413251903491, random_state=42, tol=1e-05, max_iter=10000.0 
[CV] C=5.623413251903491e-06, penalty=l2, loss=squared_hinge, dual=True, multi_class=ovr, fit_intercept=True, intercept_scaling=1000000000.0, random_state=42, tol=1e-05, max_iter=825.4041852680182 
[CV] C=0.00017782794100389227, penalty=l1, loss=hinge, dual=True, multi_class=ovr, fit_intercept=False, intercept_scaling=31622.776601683792, random_state=42, tol=1e-05, max_iter=464.15888336127773 
[CV] C=5.623413251903491, penalty=l1, loss=hinge, dual=False, multi_class=crammer_singer, fit_intercept=False, intercept_scaling=177827941.00389227, random_state=42, tol=1e-05, max_iter=562.341325190349 
[CV]  C=1e-06, penalty=l1, loss=squared_

[CV] C=5623413.251903491, penalty=l1, loss=squared_hinge, dual=True, multi_class=crammer_singer, fit_intercept=False, intercept_scaling=5.623413251903491e-09, random_state=42, tol=1e-05, max_iter=3162.2776601683795 
[CV]  C=31622776.60168379, penalty=l2, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=1.7782794100389227e-07, random_state=42, tol=1e-05, max_iter=1000.0, score=0.9805097451274363, total=   0.1s
[CV] C=31622776.60168379, penalty=l2, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=1.7782794100389227e-07, random_state=42, tol=1e-05, max_iter=1000.0 
[CV]  C=3.162277660168379e-08, penalty=l1, loss=hinge, dual=False, multi_class=crammer_singer, fit_intercept=False, intercept_scaling=0.00017782794100389227, random_state=42, tol=1e-05, max_iter=3831.1868495572853, score=0.49975012493753124, total=   0.2s
[CV] C=3.162277660168379e-08, penalty=l1, loss=hinge, dual=False, multi_class=crammer_singer, fit

[CV]  C=0.00017782794100389227, penalty=l1, loss=squared_hinge, dual=True, multi_class=crammer_singer, fit_intercept=True, intercept_scaling=31.622776601683793, random_state=42, tol=1e-05, max_iter=4641.588833612777, score=0.704647676161919, total=   0.5s
[CV] C=0.00017782794100389227, penalty=l1, loss=squared_hinge, dual=True, multi_class=crammer_singer, fit_intercept=True, intercept_scaling=31.622776601683793, random_state=42, tol=1e-05, max_iter=4641.588833612777 
[CV]  C=0.00017782794100389227, penalty=l1, loss=squared_hinge, dual=True, multi_class=crammer_singer, fit_intercept=True, intercept_scaling=31.622776601683793, random_state=42, tol=1e-05, max_iter=4641.588833612777, score=0.6731634182908546, total=   0.6s
[CV] C=0.00017782794100389227, penalty=l1, loss=squared_hinge, dual=True, multi_class=crammer_singer, fit_intercept=True, intercept_scaling=31.622776601683793, random_state=42, tol=1e-05, max_iter=4641.588833612777 
[CV]  C=5623.413251903491, penalty=l1, loss=squared_hin

...

[CV] C=5623413.251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=215.44346900318845 
[CV]  C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.005623413251903491, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9895, total=   0.4s
[CV] C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.005623413251903491, random_state=42, tol=1e-05, max_iter=316.22776601683796 
[CV]  C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9890054972513743, total=   0.5s
[CV] C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, 

[CV] C=3.1622776601683795e-05, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=31622776.60168379, random_state=42, tol=1e-05, max_iter=261.0157215682536 
[CV]  C=3.1622776601683795e-05, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=31622776.60168379, random_state=42, tol=1e-05, max_iter=261.0157215682536, score=0.49975012493753124, total=   0.0s
[CV] C=3.1622776601683795e-05, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=31622776.60168379, random_state=42, tol=1e-05, max_iter=261.0157215682536 
[CV] C=5623.413251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=True, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=261.0157215682536 
[CV]  C=177827.94100389228, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=31622776.6

[CV] C=5623.413251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=True, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=261.0157215682536 
[CV]  C=1000000.0, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.00017782794100389227, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9890054972513743, total=   0.4s
[CV] C=1000000.0, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.00017782794100389227, random_state=42, tol=1e-05, max_iter=316.22776601683796 
[CV]  C=177827.94100389228, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9895, total=   0.5s
[CV] C=177827.94100389228, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scalin

[CV] C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.005623413251903491, random_state=42, tol=1e-05, max_iter=316.22776601683796 
[CV]  C=5623413.251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.00017782794100389227, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9895, total=   0.4s
[CV]  C=31622776.60168379, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=215.44346900318845, score=0.9864932466233116, total=   0.4s
[CV] C=5623413.251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.00017782794100389227, random_state=42, tol=1e-05, max_iter=316.22776601683796 
[CV] C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=Fals

[CV]  C=5623413.251903491, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9874937468734367, total=   0.3s
[CV]  C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9824912456228114, total=   0.4s
[CV] C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796 
[CV]  C=31622.776601683792, penalty=l1, loss=squared_hinge, dual=False, multi_class=ovr, fit_intercept=False, intercept_scaling=0.03162277660168379, random_state=42, tol=1e-05, max_iter=316.22776601683796, score=0.9874937468734367, total=   0.4s
15 	29    	0.958014	0     	0.9876	0.15294   
Best individu

In [ ]:
best_indiv = LinearSVC(C = 5623.413251903491, 
                     penalty = 'l1',
                     loss = 'squared_hinge',
                     dual = False,
                     multi_class = 'ovr',
                     fit_intercept = True,
                     intercept_scaling = 0.03162277660168379,
                     random_state = 42,
                     tol = 1e-05,
                     max_iter = 316.22776601683796)